In [36]:
import torch
from torchvision import datasets, transforms
from ipywidgets import IntProgress
from PIL import Image
import numpy as np
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torch.nn as nn

In [81]:
t = transforms.Compose([transforms.ToTensor(),
                       transforms.Lambda(lambda x : x.reshape(-1))
                       ])
target_t = lambda label: F.one_hot(torch.tensor(label), num_classes=10)

train_dataset = datasets.SVHN(root="./data", split="train", transform= t, target_transform=target_t, download=True)
test_dataset = datasets.SVHN(root="./data", split="test", transform= t,target_transform=target_t, download=True)

Using downloaded and verified file: ./data/train_32x32.mat
Using downloaded and verified file: ./data/test_32x32.mat


In [212]:
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=True)

In [213]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(in_features=3072, out_features=512)
        self.layer2 = nn.Linear(in_features=512, out_features=256)
        self.layer3 = nn.Linear(in_features=256, out_features=64)
        self.layer4 = nn.Linear(in_features=64, out_features=10)
   
    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        x = F.relu(self.layer3(x))
        x = self.layer4(x)
        pred = F.log_softmax(x, dim=1)
        return pred

In [214]:
device = "cuda"
model = Network()
model.to(device)
criterion = nn.CrossEntropyLoss().to(device)

lr = 0.1
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [215]:
def training(Epochs=200):
    loss_list= []
    for epoch in range(Epochs):
        loss_epoch = 0.0
        for img, label in train_loader:
            img, label = img.to(device), label.type(torch.float32).to(device)

            pred = model.forward(img)

            loss = criterion(pred, label)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            loss_epoch += loss.item()

        loss_list.append(loss)
        print(f"Loss for epoch {epoch + 1} is {loss_epoch}")

In [216]:
training()

Loss for epoch 1 is 39429.11594390869
Loss for epoch 2 is 642.36310338974
Loss for epoch 3 is 642.4290549755096
Loss for epoch 4 is 642.7176694869995
Loss for epoch 5 is 642.7764122486115
Loss for epoch 6 is 642.9301431179047
Loss for epoch 7 is 642.822319984436
Loss for epoch 8 is 642.7927887439728
Loss for epoch 9 is 642.9536166191101
Loss for epoch 10 is 643.0875945091248
Loss for epoch 11 is 642.8581266403198
Loss for epoch 12 is 642.9057238101959
Loss for epoch 13 is 643.0232133865356
Loss for epoch 14 is 643.1456761360168
Loss for epoch 15 is 642.9479393959045


KeyboardInterrupt: 